In [ ]:
!pip uninstall -y qiskit qiskit-terra qiskit-aer qiskit-ibmq-provider 

In [ ]:
pip list 

In [ ]:
pip install qiskit

In [ ]:
pip install qiskit-aer

In [ ]:
import qiskit
print(qiskit.__version__) 

In [ ]:
pip install qiskit-ibm-runtime 

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService


QiskitRuntimeService.save_account(
    channel="ibm_quantum",
    token="fa8b4d69aa9bb58681f89c9e419fb58b5e77a03c882e5d0239cb118155de540f7cb72a526f79c579e93217777de8ade16e4772a06ae265b2d3350ed89e69e1fc",
    overwrite=True
)


service = QiskitRuntimeService()
print(service.backends())

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler
from qiskit.exceptions import QiskitError  # General Qiskit exception

class QuantumResourceAuction:
    def __init__(self, resources, users, bids):
        self.resources = resources
        self.users = users
        self.bids = bids
        self.allocations = {}
        self.payments = {user: 0 for user in users}

    def run_auction(self):
        for r in self.resources:
            highest_bid = -float('inf')
            second_highest_bid = -float('inf')
            winner = None
            
            # Determine the highest and second-highest bids
            for u in self.users:
                bid = self.bids[u].get(r, 0)
                if bid > highest_bid:
                    second_highest_bid = highest_bid
                    highest_bid = bid
                    winner = u
                elif bid > second_highest_bid:
                    second_highest_bid = bid
            
            # Allocate the resource to the highest bidder if there is a valid bid
            if winner is not None and highest_bid > 0:
                self.allocations[r] = winner
                # Set the payment as the second-highest bid
                self.payments[winner] += second_highest_bid

    def print_results(self):
        print("Resource Allocations:")
        for r, u in self.allocations.items():
            print(f"Resource {r} is allocated to User {u}")
        
        print("\nUser Payments:")
        for u, payment in self.payments.items():
            print(f"User {u} pays {payment}")

    def run_quantum_job(self):
        # Initialize Qiskit Runtime Service
        try:
            service = QiskitRuntimeService()
        except QiskitError as e:
            print(f"Error initializing Qiskit Runtime Service: {e}")
            return

        # Try to select the least busy backend with at least 5 qubits that is not a simulator
        try:
            backends = service.backends(filters=lambda x: x.configuration().n_qubits >= 5 and not x.configuration().simulator)
            if not backends:
                raise QiskitError("No suitable backend found.")
            backend = sorted(backends, key=lambda b: b.status().pending_jobs)[0]  # Select the least busy backend
        except QiskitError as e:
            print(f"Error: {e}")
            return

        print(f"Selected Backend: {backend.name}")

        # Create a simple quantum circuit
        qc = QuantumCircuit(5, 5)
        qc.h(0)
        qc.cx(0, 1)
        qc.cx(0, 2)
        qc.cx(0, 3)
        qc.cx(0, 4)
        qc.measure(range(5), range(5))

        # Transpile the circuit for the selected backend
        transpiled_qc = transpile(qc, backend=backend)

        # Run the quantum job in a runtime session
        with Session(backend=backend) as session:
            sampler = Sampler(mode=session)  # Use mode=session to run in session mode
            result = sampler.run([transpiled_qc]).result()

        # Extract and print the results
        try:
            counts = result.get('counts', None)  # Adjust this line based on available attributes
            if counts is None:
                raise ValueError("No counts found in the result.")
            print("Job Result:", counts)
        except Exception as e:
            print(f"Error extracting results: {e}")

# Test Cases

# Test 1: Simple Case
print("Test 1: Simple Case")
resources = ['Qubit1', 'Qubit2', 'Qubit3']
users = ['UserA', 'UserB', 'UserC']

bids = {
    'UserA': {'Qubit1': 10, 'Qubit2': 15, 'Qubit3': 5},
    'UserB': {'Qubit1': 20, 'Qubit2': 10, 'Qubit3': 25},
    'UserC': {'Qubit1': 30, 'Qubit2': 35, 'Qubit3': 10}
}

auction = QuantumResourceAuction(resources, users, bids)
auction.run_auction()
auction.print_results()

# Test 2: Case with Tied Bids
print("\nTest 2: Case with Tied Bids")
resources = ['Qubit1', 'Qubit2', 'Qubit3']
users = ['UserA', 'UserB', 'UserC']

bids = {
    'UserA': {'Qubit1': 10, 'Qubit2': 15, 'Qubit3': 20},
    'UserB': {'Qubit1': 10, 'Qubit2': 15, 'Qubit3': 20},
    'UserC': {'Qubit1': 30, 'Qubit2': 35, 'Qubit3': 10}
}

auction = QuantumResourceAuction(resources, users, bids)
auction.run_auction()
auction.print_results()

# Test 3: Case with No Bids for a Resource
print("\nTest 3: Case with No Bids for a Resource")
resources = ['Qubit1', 'Qubit2', 'Qubit3']
users = ['UserA', 'UserB', 'UserC']

bids = {
    'UserA': {'Qubit1': 0, 'Qubit2': 0, 'Qubit3': 0},
    'UserB': {'Qubit1': 20, 'Qubit2': 10, 'Qubit3': 0},
    'UserC': {'Qubit1': 0, 'Qubit2': 0, 'Qubit3': 10}
}

auction = QuantumResourceAuction(resources, users, bids)
auction.run_auction()
auction.print_results()

auction.run_quantum_job()
